**Practica de BIG DATA PROCESSING**

El objetivo de la presente práctica es contestar a las siguientes pregutnas:

1. ¿Cuál es el país más “feliz” del 2021 según la data? (considerar que la columna “Ladder score” mayor número más feliz es el país)
2. ¿Cuál es el país más “feliz” del 2021 por continente según la data?
3. ¿Cuál es el país que más veces ocupó el primer lugar en todos los años?
4. ¿Qué puesto de Felicidad tiene el país con mayor GDP del 2020?
5. ¿En que porcentaje a variado a nivel mundial el GDP promedio del 2020 respecto al 2021? ¿Aumentó o disminuyó?
6. ¿Cuál es el país con mayor expectativa de vide (“Healthy life expectancy at birth”)? Y ¿Cuánto tenia en ese indicador en el 2019?


Para ello en primer lugar y con el fin de facilitar el proceso, vamos a realizar una homogeneización de campos de ambos datasets.

Del resultado del análisis de las cuestiones derivamos que es necesario un DataSet con los siguientes campos: Pais, Año, Indice de Felicidad, PIB o GDP per capita, Expectativa de vida y Continente


**Construimos el DF conjunto**

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
wh2021 = pd.read_csv("/content/drive/MyDrive/Master Big_Data/big-data-processing/PROYECTO FINAL/Datasets/world-happiness-report-2021.csv")
whR = pd.read_csv("/content/drive/MyDrive/Master Big_Data/big-data-processing/PROYECTO FINAL/Datasets/world-happiness-report.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
whR = whR[['Country name','year','Life Ladder','Log GDP per capita','Healthy life expectancy at birth']]
whR = whR.set_axis(['Pais','Año','Indice_Felicidad','PIB','Esperanza_Vida'], axis=1)
whR.head(5)



,Pais,Año,Indice_Felicidad,PIB,Esperanza_Vida
0,Afghanistan,2008,3.724,7.370,50.80
1,Afghanistan,2009,4.402,7.540,51.20
2,Afghanistan,2010,4.758,7.647,51.60
3,Afghanistan,2011,3.832,7.620,51.92
4,Afghanistan,2012,3.783,7.705,52.24


In [26]:
wh2021R = wh2021[['Country name','Ladder score','Logged GDP per capita','Healthy life expectancy','Regional indicator']]
wh2021R.insert(1,'Year',2021)
wh2021R = wh2021R.set_axis(['Pais','Año','Indice_Felicidad','PIB','Esperanza_Vida','Continente'], axis=1)
wh2021R.head(5)


,Pais,Año,Indice_Felicidad,PIB,Esperanza_Vida,Continente
0,Finland,2021,7.842,10.775,72.0,Western Europe
1,Denmark,2021,7.620,10.933,72.7,Western Europe
2,Switzerland,2021,7.571,11.117,74.4,Western Europe
3,Iceland,2021,7.554,10.878,73.0,Western Europe
4,Netherlands,2021,7.464,10.932,72.4,Western Europe


In [42]:
Union = pd.concat([wh2021R,whR]).reset_index(drop=True)
Union


,Pais,Año,Indice_Felicidad,PIB,Esperanza_Vida,Continente
0,Finland,2021,7.842,10.775,72.0,Western Europe
1,Denmark,2021,7.620,10.933,72.7,Western Europe
2,Switzerland,2021,7.571,11.117,74.4,Western Europe
3,Iceland,2021,7.554,10.878,73.0,Western Europe
4,Netherlands,2021,7.464,10.932,72.4,Western Europe
...,...,...,...,...,...,...
2093,Zimbabwe,2016,3.735,7.984,54.4,NaN
2094,Zimbabwe,2017,3.638,8.016,55.0,NaN
2095,Zimbabwe,2018,3.616,8.049,55.6,NaN
2096,Zimbabwe,2019,2.694,7.950,56.2,NaN


**### Pregunta 1**
¿Cuál es el país más “feliz” del 2021 según la data? (considerar que la columna “Ladder score” mayor número más feliz es el país)

In [28]:
Question1 = Union.query('Año == 2021').nlargest(1,'Indice_Felicidad')
Question1

,Pais,Año,Indice_Felicidad,PIB,Esperanza_Vida,Continente
0,Finland,2021,7.842,10.775,72.0,Western Europe


### Pregunta 2
¿Cuál es el país más “feliz” del 2021 por continente según la data?

In [51]:
Question2=Union.query('Año == 2021')
Mas_Felices= Question2.groupby('Continente')['Indice_Felicidad'].idxmax()
Question2 = Question2.loc[Mas_Felices]
Question2[['Continente','Pais','Indice_Felicidad']].sort_values(by='Indice_Felicidad', ascending=False)

,Continente,Pais,Indice_Felicidad
0,Western Europe,Finland,7.842
8,North America and ANZ,New Zealand,7.277
11,Middle East and North Africa,Israel,7.157
15,Latin America and Caribbean,Costa Rica,7.069
17,Central and Eastern Europe,Czech Republic,6.965
23,East Asia,Taiwan Province of China,6.584
31,Southeast Asia,Singapore,6.377
41,Commonwealth of Independent States,Uzbekistan,6.179
49,Sub-Saharan Africa,Mauritius,6.049
86,South Asia,Nepal,5.269


### Pregunta 3
¿Cuál es el país que más veces ocupó el primer lugar en todos los años?

In [69]:
ranking = Union.groupby(['Año'])['Indice_Felicidad'].idxmax()
Question3=Union.loc[ranking].groupby('Pais').count()
Question3 = Question3.sort_values(by='Indice_Felicidad', ascending=False)
Question3[['Año']]

,Año
Pais,
Denmark,7
Finland,7
Canada,1
Norway,1
Switzerland,1


### Pregunta 4
¿Qué puesto de Felicidad tiene el país con mayor GDP del 2020?

In [71]:
Question4 = Union.query('Año == 2020').sort_values(by =['PIB'], ascending=False)
Question4['rank'] = Question4['Indice_Felicidad'].rank()
Question4[['Pais','PIB','rank']].head(1)



,Pais,PIB,rank
948,Ireland,11.323,83.0


### Pregunta 5
¿En que porcentaje a variado a nivel mundial el GDP promedio del 2020 respecto al 2021? ¿Aumentó o disminuyó?

In [74]:
prom_2020 = Union.query('Año == 2020').PIB.mean()
prom_2021 = Union.query('Año == 2021').PIB.mean()
Question5 = ((prom_2021 - prom_2020) / prom_2020) * 100
Question5

-3.272594678251047

### Pregunta 6
¿Cuál es el país con mayor expectativa de vida (“Healthy life expectancy at birth”)? Y ¿Cuánto tenia en ese indicador en el 2019?

In [101]:

MAX_VIDA = Union.query("Año == 2021")["Esperanza_Vida"].idxmax()
Question6 = Union.iloc[MAX_VIDA][["Pais", "Año", "Esperanza_Vida"]]

query_pais =  Question6["Pais"]


In [110]:

Question6_bis= Union.query("Año == 2019 and Pais == @query_pais")
Question6_bis


,Pais,Año,Indice_Felicidad,PIB,Esperanza_Vida,Continente
1682,Singapore,2019,6.378,11.486,77.1,NaN
